<a href="https://colab.research.google.com/github/SEEsuite/colab_scripts/blob/dev/reddit_edgelist_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Oct 22, 2022

Take reddit data and get "edge list" and a few other things

Edge list is a graphs where nodes are users and edges are the interactions between them.


In [ ]:
import pandas as pd
import os
import numpy as np
import datetime
import time
import math
import json
import requests
import itertools
import numpy as np
import time
from datetime import datetime, timedelta

In [ ]:
def get_time(my_time):
    time_str = my_time
    utc_time = datetime.datetime.strptime(time_str, '%Y-%m-%d %H:%M:%S.%f')
    a = time.mktime(datetime.datetime.strptime(time_str,
                                             '%Y-%m-%d %H:%M:%S.%f').timetuple())

    print(a)
    return int(a)
    

In [ ]:
import urllib.request
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

def import_data_from_drive(share_link, your_name_for_file="my_data"):
  """Brings data file from a google drive sharepoint to your colab workspace.
     It does not require you to host the dataset on your own account.

     Parameters:
     share_link: the link to view a file in google drive
     our_name_for_file: a string describing the file, preferable endling in a file type, ex. 'data.csv'
     """
  id = share_link.split("/")[5] # separate the id from the link
  print("Using id", id, "to find file on drive")

  # use pydrive and colab modules to authenticate you
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  print("Authenticated colab user")

  # This step will move the file from Drive to the workspace
  download_object = drive.CreateFile({'id':id}) 
  download_object.GetContentFile(your_name_for_file)
  print("Added file to workspace with name", your_name_for_file)

  return

## Load data from folder of files
If the data is scattered, pass in the path to the single folder containing data.

In [ ]:
# !unzip /content/RedditData2022.zip

In [ ]:
#CHANGE THIS
directory = '/content/reddit_data'

o_df = pd.DataFrame()
print( os.listdir(directory))
for f in os.listdir(directory):
    if '$' in f:
        continue
    file = os.path.join(directory, f)
    print(file)
    sub_df = pd.read_excel(file, header=7)
    assert 'Full Text' in sub_df.columns
    o_df = pd.concat((o_df, sub_df), axis=0)

['mentions (35).xlsx', 'mentions (34).xlsx', 'mentions (44).xlsx', 'mentions (40).xlsx', 'mentions (43).xlsx', 'mentions (42).xlsx', 'mentions (36).xlsx', 'mentions (38).xlsx', 'mentions (41).xlsx', 'mentions (39).xlsx', 'mentions (37).xlsx']
/content/reddit_data/mentions (35).xlsx
/content/reddit_data/mentions (34).xlsx
/content/reddit_data/mentions (44).xlsx
/content/reddit_data/mentions (40).xlsx
/content/reddit_data/mentions (43).xlsx
/content/reddit_data/mentions (42).xlsx
/content/reddit_data/mentions (36).xlsx
/content/reddit_data/mentions (38).xlsx
/content/reddit_data/mentions (41).xlsx
/content/reddit_data/mentions (39).xlsx
/content/reddit_data/mentions (37).xlsx


# Load data from single excel sheet

In [ ]:

link = "https://docs.google.com/spreadsheets/d/1m1-qV00Qkm2m9Znypj_ORBZgAQ9yQ9eO/edit?usp=sharing&ouid=101042095541764641159&rtpof=true&sd=true"
import_data_from_drive(link, your_name_for_file="tweets.xlsx")
df = pd.read_excel('tweets.xlsx')

Archive:  /content/RedditData2022.zip
mapname:  conversion of  failed
 extracting: mentions (44).xlsx      
 extracting: mentions (35).xlsx      
 extracting: mentions (41).xlsx      
 extracting: mentions (43).xlsx      
 extracting: mentions (36).xlsx      
 extracting: mentions (37).xlsx      
 extracting: mentions (38).xlsx      
 extracting: mentions (39).xlsx      
 extracting: mentions (40).xlsx      
 extracting: mentions (42).xlsx      
 extracting: mentions (34).xlsx      


# Preprocess

In [ ]:
def do(x):
    if '2022-09-11' in x or '2022-08-31' in x:
        return pd.NA
    else:
        return x

In [ ]:
df = o_df.copy()

In [ ]:
print(len(df), "instances")
df['Date'] = df['Date'].apply(do)
df = df.dropna(subset=['Date'])
print(len(df), "instances")

34579 instances
31277 instances


In [ ]:
# remove hard duplicates
crop_df = df.drop_duplicates(subset=["Full Text"], ignore_index=True).copy()
len(crop_df)

28140

In [ ]:
# this block is borrowed from tutorial
def make_request(uri, max_retries = 2):
    def fire_away(uri):
        response = requests.get(uri)
        print(response.status_code)
        assert response.status_code == 200
        return json.loads(response.content)
    current_tries = 1
    while current_tries < max_retries:
        try:
            time.sleep(1.1)
            response = fire_away(uri)
            return response
        except:
            time.sleep(1.1)
            current_tries += 1
    return fire_away(uri)

In [ ]:
def  get_ids(comment_id):
    url = 'https://api.pushshift.io/reddit/comment/search?ids=' + comment_id
#     print(url)
    ret = {'parent_id': None, 'link_id': None, 'body': None, 'author':None, 'author_flair': None}
    j = make_request(url)
    j = j['data'][0]
    ret['body'] = j['body']
    ret['link_id'] = np.base_repr(int(j['link_id']), base=10)
    ret['parent_id'] = j['parent_id']
    ret['author'] = j['author']
    ret['author_flair'] = j['author_flair_text']
#     print(ret['parent_id'])
    
    return ret

def get_poster(sub_id):
#     parent_id = np.base_repr(int(sub_id), base=36)
    new_url = 'https://api.pushshift.io/reddit/submission/search?ids=' +sub_id
    print("NEW", new_url)
    j = make_request(new_url)
    j = j['data'][0]
    auth = j['author']
    body = j['title']
    flair = None
    if 'author_flair_text' in j.keys():
        flair = j['author_flair_text']
    elif'author_flair_richtext' in j.keys():
        flair = j['author_flair_richtext']
    return auth, body, flair

def get_parent_author(parent_id):
    parent_id = np.base_repr(int(parent_id), base=10)
    new_url = 'https://api.pushshift.io/reddit/comment/search?ids=' + parent_id
    j = make_request(new_url)
    j = j['data'][0]
    auth = j['author']
    body = j['body']
    flair = None
    if 'author_flair_text' in j.keys():
        flair = j['author_flair_text']
    return auth, body, flair


    

In [ ]:
def my_parse(x):
#     print(x)
    url = x
    author = None
    parent_author = None
    comment_id = None
    parent_id = None 
    link_id = None 
    interaction_type = 'comment'
    author_flair = None
    parent_author_flair = None
    body= None
    parent_body= None
    failure = False
    

    parse = url.split('/')
    parse = [p for p in parse if p != '']
#     print(parse)
    assert parse[4] == 'comments'
    comment_id = parse[-1]
    parse = parse[5:]

    if len(parse) == 2:
#         print("self")
        interaction_type = 'post'

        
#         author, body, flair = get_poster(comment_id)

    elif len(parse) >2:
        d = get_ids(comment_id)
        body = d['body']
        author = d['author']
        author_flair = d['author_flair']
        if not d['parent_id']:
#             print("1st comment")
            link_id = d['link_id']
            parent_author, parent_body, parent_flair = get_poster(link_id)
            pass
        else:
#             print("nth comment")
            parent_author, parent_body, parent_author_flair = get_parent_author(d['parent_id'])
#     except:
#         print("Failed on", url)
#         failure = True
    x = [author, parent_author, comment_id, parent_id, link_id, interaction_type, author_flair, parent_author_flair, body, parent_body, url]
    return x


In [ ]:
series = crop_df['Url']
series.apply(my_parse)